In [ ]:
%matplotlib notebook

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
from IPython.display import Image
from IPython.display import Math
from ipywidgets import interact, Dropdown
from IPython.display import display

import os
import sys
import shutil
import pickle
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import geopandas as gpd
import flopy as fp
import fit_parametric_distributions
import pandas as pd
import gdal
from scipy.interpolate import griddata
from shapely.geometry import Point, LineString
from flopy.export import vtk as vt

In [ ]:
homes = ['../Models']
fig_dir = '../Figures'

mfpth = '../executables/MODFLOW-NWT_1.0.9/bin/MODFLOW-NWT_64.exe'
mp_exe_name = '../executables/modpath.6_0/bin/mp6.exe' 

mf_start_date_str = '01/01/1900' 
mp_release_date_str = '01/01/2020' 

por = 0.20

weight_scheme = 'flow'
# weight_scheme = 'volume'

# set the number of evenly spaced vertical sections to plot  
num_row_sections = 3
num_col_sections = 3
layers_to_plot = [0, 2]

# shapefile name and color in a tuple
# shapefiles_to_plot = [('domain_outline.shp', 'none'), ('NHD_clip.shp', 'blue')]
shapefiles_to_plot = [('domain_outline.shp', 'none')]

# height of each section in the figures, in inches
ht_per_section = 2

general_model = True

Select the model to process

In [ ]:
dir_list = []
mod_list = []
i = 0

for home in homes:
    if os.path.exists(home):
        for dirpath, dirnames, filenames in os.walk(home):
            for f in filenames:
                if os.path.splitext(f)[-1] == '.nam':
                    mod = os.path.splitext(f)[0]
                    mod_list.append(mod)
                    dir_list.append(dirpath)
                    i += 1
print('    {} models read'.format(i))

model_area = Dropdown(
    options=mod_list,
    description='Model:',
    background_color='cyan',
    border_color='black',
    border_width=2)
display(model_area)

with open('dir_list.txt', 'w') as f:
    for i in dir_list:
        f.write('{}\n'.format(i))

In [ ]:
model = model_area.value
model_ws = [item for item in dir_list if model in item][0]
nam_file = '{}.nam'.format(model)
print("working model is {}".format(model_ws))

Select a GeoTiff that contains information about the model grid (origin and rotation) and an array with land surface altitude.  If one does not exist, there is a notebook that can create on from a shapefile of the grid oultine. 

In [ ]:
tif_list = []
tif_dir_list = []
t_index = 0

for home in homes:
    if os.path.exists(home):
        for dirpath, dirnames, filenames in os.walk(home):
            for f in filenames:
                if '.tif' in f:
                    tif = os.path.splitext(f)[0]
                    tif_list.append(os.path.join(dirpath, tif))
                    tif_dir_list.append(os.path.join(dirpath, f))
                    t_index += 1
print('    {} models read'.format(i))

tif_file = Dropdown(
    options=tif_list,
    description='GeoTiff files:',
    background_color='cyan',
    border_color='black',
    border_width=2)
display(tif_file)

with open('tif_list.txt', 'w') as f:
    for i in dir_list:
        f.write('{}\n'.format(i))

In [ ]:
modelgrid_tif = tif_file.value
tif_src = [item for item in tif_dir_list if modelgrid_tif in item][0]

print("Model grid GeoTiff file is {}".format(tif_src))

In [ ]:
# for pth in dir_list:
#     model = os.path.normpath(pth).split(os.sep)[2]
#     model_ws = [item for item in dir_list if model in item][0]
#     nam_file = '{}.nam'.format(model)
#     print("working model is {}".format(model_ws))

In [ ]:
print ('Reading model information')

fpmg = fp.modflow.Modflow.load(nam_file, model_ws=model_ws, exe_name=mfpth, version='mfnwt', 
                               load_only=['DIS', 'BAS6', 'UPW', 'OC'], check=False)

dis = fpmg.get_package('DIS')
bas = fpmg.get_package('BAS6')
upw = fpmg.get_package('UPW')
oc = fpmg.get_package('OC')

delr = dis.delr
delc = dis.delc
nlay = dis.nlay
nrow = dis.nrow
ncol = dis.ncol
bot = dis.getbotm()
top = dis.gettop()
# hk = upw.hk.get_value()

hnoflo = bas.hnoflo
ibound = np.asarray(bas.ibound.get_value())
hdry = upw.hdry

if general_model:
    top /= 1.2

print ('   ... done') 

Make list of vertical sections.

In [ ]:
# setup dictionaries of the MODFLOW units for proper labeling of figures.
lenunit = {0:'undefined units', 1:'feet', 2:'meters', 3:'centimeters'}
timeunit = {0:'undefined', 1:'second', 2:'minute', 3:'hour', 4:'day', 5:'year'}

# Create dictionary of multipliers for converting model time units to days
time_dict = dict()
time_dict[0] = 1.0 # undefined assumes days, so enter conversion to days
time_dict[1] = 24 * 60 * 60
time_dict[2] = 24 * 60
time_dict[3] = 24
time_dict[4] = 1.0
time_dict[5] = 1.0

In [ ]:
# convert string representation of dates into Python datetime objects
mf_start_date = dt.datetime.strptime(mf_start_date_str , '%m/%d/%Y')
mp_release_date = dt.datetime.strptime(mp_release_date_str , '%m/%d/%Y')

# convert simulation time to days from the units specified in the MODFLOW DIS file
sim_time = np.append(0, dis.get_totim())
sim_time /= time_dict[dis.itmuni]

# make a list of simulation time formatted as calendar dates
date_list = [mf_start_date + dt.timedelta(days = item) for item in sim_time]

# reference time and date are set to the end of the last stress period
ref_time = sim_time[-1]
ref_date = date_list[-1]

# release time is calculated in tracking time (for particle release) and 
# in simulation time (for identifying head and budget components)
release_time_trk = np.abs((ref_date - mp_release_date).days)
release_time_sim = (mp_release_date - mf_start_date).days

In [ ]:
src = os.path.join(tif_src)
nf = gdal.Open(src)
band = nf.GetRasterBand(1)
land_surface = band.ReadAsArray()
gt = nf.GetGeoTransform()
proj = nf.GetProjection()
nf = None

In [ ]:
src = os.path.join(model_ws, fpmg.namefile)
name_file_df = pd.read_table(src, header=None, comment='#', delim_whitespace=True, 
              names=['package', 'unit', 'filename', 'type'])

name_file_df['package'] = name_file_df.package.str.lower()
name_file_df.set_index('unit', inplace=True)

head_file_name = name_file_df.loc[oc.iuhead, 'filename']
bud_file_name = name_file_df.loc[oc.get_budgetunit(), 'filename']

In [ ]:
src = os.path.join(model_ws, head_file_name)
hd_obj = fp.utils.HeadFile(src)
head_df = pd.DataFrame(hd_obj.recordarray)

heads = hd_obj.get_data(kstpkper=(0, 0))

heads[heads == hnoflo] = np.nan

with np.errstate(invalid='ignore'):
    heads[heads <= hdry] = np.nan
with np.errstate(invalid='ignore'):
    heads[heads > 1E+29] = np.nan

hin = np.argmax(np.isfinite(heads), axis=0)
row, col = np.indices((hin.shape))
water_table = heads[hin, row, col]

In [ ]:
# Transform GWPs into fractional row, column coordinates for each model    
# format the geotransformation list into an affine transformation matrix
forward_transform = np.array(gt).reshape(2, -1)
# add a row to get homogeneous coodinates (offsets are in the first column)
forward_transform = np.vstack((forward_transform, [1, 0, 0]))

# reverse transform cell-center coordinates to projected coordinates
r, c = np.indices(water_table.shape)
dum = np.column_stack((np.ones_like(c.ravel()), c.ravel() + 0.5, r.ravel() + 0.5)) 
cell_center = forward_transform.dot(dum.T).T
cell_center_x = cell_center[:, 0].reshape(nrow, ncol)
cell_center_y = cell_center[:, 1].reshape(nrow, ncol)

dum = np.column_stack((np.ones_like(c.ravel()), c.ravel(), r.ravel())) 
grid_verts = forward_transform.dot(dum.T).T
grid_verts_x = grid_verts[:, 0].reshape(nrow, ncol)
grid_verts_y = grid_verts[:, 1].reshape(nrow, ncol)

In [ ]:
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 8,
        'sans-serif' : 'Arial'}

plt.rc('font', **font)

In [ ]:
try:
    proj_name = [item for item in proj.split(',') if 'PROJECTION' in item][0].split('"')[1]
except:
    proj_name = ''
try:
    datum = [item for item in proj.split(',') if 'DATUM' in item][0].split('"')[1]
except:
    datum = ''
try:
    spheroid = [item for item in proj.split(',') if 'SPHEROID' in item][0].split('"')[1]
except:
    spheroid = ''
try:
    map_units = [item for item in proj.split(',') if 'UNIT' in item][1].split('"')[1]
except:
    map_units = ''

proj_str = '{} {}\n{} {}'.format(proj_name, spheroid, datum, map_units)

In [ ]:
def plot_sections_filled(ax, section, xplot, direction, num):
    colors = plt.cm.nipy_spectral(np.linspace(0, 1, nlay))

    ax.plot(xplot, top[section], label='land surface', color='black', lw=0.5)
    ax.plot(xplot, water_table[section], label='simulated\nwater table', color='blue', lw=0.75)
    ax.fill_between(xplot, top[section], bot[0][section], alpha=0.25, 
             color='blue', lw=0.75, label='model layer')
    for lay in range(dis.nlay - 1):
        ax.fill_between(xplot, bot[lay][section], bot[lay+1][section],   
                        color=colors[lay], alpha=0.250, lw=0.75, linestyle='None')
    ax.set_title('{} {}'.format(direction, num), position=(0.5, 0.85), ha='center')

def plot_sections(ax, section, xplot, direction, num):
    colors = plt.cm.nipy_spectral(np.linspace(0, 1, nlay))

    ax.plot(xplot, top[section], label='land surface', color='black', lw=0.5)
    ax.plot(xplot, water_table[section], label='simulated\nwater table', 
               color='blue', lw=0.75)
    ax.plot(xplot, bot[0][section], alpha=0.25, 
               color='blue', lw=0.75, label='model layer')
    for lay_array in bot[1:,:,:]:
        ax.plot(xplot, lay_array[section], 
                        color='black', alpha=0.250, lw=0.75, linestyle='solid')
    ax.set_title('{} {}'.format(direction, num), position=(0.5, 0.85), ha='center')
    
def make_plot_coordinates(water_table, bot, cell_center, section, n, num_ordinates):
    xplot = cell_center[:, 0].reshape(nrow, ncol)[section]
    yplot = cell_center[:, 1].reshape(nrow, ncol)[section]

    wat_tab_xs = water_table[section]    
    bot_xs = bot[-1][section]
    tot_sat_thk = (wat_tab_xs - bot_xs).reshape(1, num_ordinates)
    frac_thk = np.linspace(0, 1, n, endpoint=True).reshape(n, 1)

    x_coord = xplot * np.ones((n, num_ordinates))
    y_coord = yplot * np.ones((n, num_ordinates))
    z_coord = bot_xs + frac_thk * tot_sat_thk
    
    xi = np.zeros((n * num_ordinates, 3))
    xi[:, 0] = x_coord.ravel()
    xi[:, 1] = y_coord.ravel()
    xi[:, 2] = z_coord.ravel()  
    return xi

def get_particle_points(ep_data, num, xs_label):
    points = ep_data.loc[ep_data[xs_label] == (num + 1), ['Initial Geog X', 
                                                            'Initial Geog Y', 
                                                            'Initial Global Z']].values    
    values = ep_data.loc[ep_data[xs_label] == (num + 1), 'rt'].values
    return points, values

def interpolate_travel_times(points, values, xi):
    return griddata(points, values, xi, method='linear')
    
def plot_travel_times(ax, x, y, tt, shp):
    with np.errstate(invalid='ignore'):
        return ax.contourf(x.reshape(shp), y.reshape(shp), tt[:].reshape(shp), 
                           colors=colors, alpha=0.5, levels=levels)    

def anno_sections(ax, num_sec, plot):
    ax.legend(loc=0, frameon=False, ncol=1)#, bbox_to_anchor=(1.0, 0.5))
    ax.set_xlabel('{}'.format(proj_str), ha='right', position=(1.0, 0.1), fontsize=6)
    if plot == np.floor_divide(num_sec, 2):
        ax.set_ylabel('Altitude in {}'.format(length_units))

def anno_age(fig, im):
    fig.colorbar(im, cax=cbar_ax, orientation='horizontal', alpha=0.5)
    cbar_ax.set_xlabel('Particle travel time in years', y=0.05, rotation=0,  ha='center')

def make_sect_ar(dim, num_sections):
    xs = slice(None) 
    ns = dim / num_sections
    n = np.arange(num_sections) + 1
    return np.int32(np.around(ns * n - (ns / 2)))

In [ ]:
length_units = lenunit[dis.lenuni]
total_col_ht = num_col_sections * ht_per_section
total_row_ht = num_row_sections * ht_per_section

levels = [0, 5, 10, 50, 100, 500, 1000, 5000, 10000]
colors = plt.cm.nipy_spectral(np.linspace(0, 1, len(levels)))
n = 1000 # number of points to interpolate in a vertical column at cell center
form = 'png'
# form = 'pdf'

if num_col_sections > 0:

    fig, axs = plt.subplots(num_col_sections, 1, figsize=(7.4, total_col_ht), sharex=True, 
                             sharey=True, gridspec_kw={'wspace':0.05, 'hspace':0.0})
    fig.subplots_adjust(top=0.95, bottom=0.2, hspace=0)

    var = make_sect_ar(ncol, num_col_sections)
    sec_list = [(slice(None), item) for item in var]

    for plot, ax in enumerate(axs):
        section = sec_list[plot]
        xplot = cell_center_y[section]
        plot_sections_filled(ax, section, xplot, 'column', section[1])
        anno_sections(ax, num_col_sections, plot)

    line = '{}_column_sections.{}'.format(model, form)
    fig_name = os.path.join(fig_dir, line)
    fig.savefig(fig_name)
#     plt.close()

if num_row_sections > 0:

    fig, axs = plt.subplots(num_row_sections, 1, figsize=(7.4, total_row_ht), sharex=True, 
                             sharey=True, gridspec_kw={'wspace':0.05, 'hspace':0.0}) 
    fig.subplots_adjust(top=0.95, bottom=0.2, hspace=0)

    var = make_sect_ar(nrow, num_row_sections)
    sec_list = [(item, slice(None)) for item in var]

    for plot, ax in enumerate(axs):
        section = sec_list[plot]
        xplot = cell_center_x[section]
        plot_sections_filled(ax, section, xplot, 'row', section[0])
        anno_sections(ax, num_col_sections, plot)

    line = '{}_row_sections.{}'.format(model, form)
    fig_name = os.path.join(fig_dir, line)
    fig.savefig(fig_name)
    #     plt.close()


Read modified endpoint file

In [ ]:
# form the path to the endpoint file
mpname = '{}_{}_all_zones_mod'.format(fpmg.name, weight_scheme)
endpoint_file = '{}.{}'.format(mpname, 'mpend')
endpoint_file = os.path.join(model_ws, endpoint_file)

# read the endpoint data
ep_data = pd.read_csv(endpoint_file, header=0, usecols=['initial_node_num', 
       'Initial Layer', 'Initial Row', 'Initial Column', 'Initial Zone',
       'Initial Local X', 'Initial Local Y', 'Initial Local Z', 'Initial Global Z',
       'Final Layer', 'Final Row', 'Final Column', 'Final Cell Face', 
        'Final Zone', 'Final Local X', 'Final Local Y', 'Final Local Z', 
       'Label', 'final_node_num', 'rt', 'xy_path_len', 'xyz_path_len'])

# add geographic coordinates to the endpoint file
ep_data['initial_model_x'] = ep_data['Initial Column'] + ep_data['Initial Local X'] - 1
ep_data['initial_model_y'] = ep_data['Initial Row'] - ep_data['Initial Local Y'] 
ep_data['initial_model_z'] = ep_data['Initial Layer'] - ep_data['Initial Local Z'] 

ep_data['final_model_x'] = ep_data['Final Column'] - 1 + ep_data['Final Local X']
ep_data['final_model_y'] = ep_data['Final Row'] - ep_data['Final Local Y'] 
ep_data['final_model_z'] = ep_data['Final Layer'] - ep_data['Final Local Z'] 

ep_data['ones'] = 1

ep_data.where(ep_data.rt < 10000, 10000, inplace=True)

ep_data.drop(['Initial Local X', 'Initial Local Y', 'Initial Local Z', 
              'Final Local X', 'Final Local Y','Final Local Z'], axis='columns', inplace=True)

dum = ep_data.loc[:, ['ones', 'initial_model_x', 'initial_model_y']]
points = forward_transform.dot(dum.T).T

ep_data['Initial Geog X'] = points[:, 0]
ep_data['Initial Geog Y'] = points[:, 1]

In [ ]:
length_units = lenunit[dis.lenuni]
total_col_ht = num_col_sections * ht_per_section
total_row_ht = num_row_sections * ht_per_section

levels = [0, 5, 10, 50, 100, 500, 1000, 5000, 10000]
colors = plt.cm.nipy_spectral(np.linspace(0, 1, len(levels)))
n = 1000 # number of points to interpolate in a vertical column at cell center
form = 'png'
# form = 'pdf'

gdf = gpd.GeoDataFrame()

if num_col_sections > 0:

    fig, axs = plt.subplots(num_col_sections, 1, figsize=(7.4, total_col_ht), sharex=True, 
                             sharey=True, gridspec_kw={'wspace':0.05, 'hspace':0.0})
    fig.subplots_adjust(top=0.95, bottom=0.2, hspace=0)
    cbar_ax = fig.add_axes([0.15, 0.12, 0.50, 0.025])

    var = make_sect_ar(ncol, num_col_sections)
    sec_list = [(slice(None), item) for item in var]

    for plot, ax in enumerate(axs):
        section = sec_list[plot]
        xplot = cell_center_y[section]
        plot_sections(ax, section, xplot, 'column', section[1])

        xi = make_plot_coordinates(water_table, bot, cell_center, section, n, xplot.shape[0])
        points, values = get_particle_points(ep_data, section[1], 'Initial Column')
        tt = interpolate_travel_times(points, values, xi)
        im = plot_travel_times(ax, xi[:,1], xi[:,2], tt, (n, nrow))
        anno_sections(ax, num_col_sections, plot)
        anno_age(fig, im)

        xline = LineString([Point(xi[0, 0], xi[0, 1]), Point(xi[-1, 0], xi[-1, 1])])
        xs_line = gpd.GeoDataFrame(geometry = gpd.GeoSeries(xline))#, crs=streams.crs)
        xs_line['section'] = 'column {}'.format(section[1])
        gdf = gdf.append(xs_line)

    line = '{}_column_tt_sections.{}'.format(model, form)
    fig_name = os.path.join(fig_dir, line)
    fig.savefig(fig_name)
#     plt.close()

if num_row_sections > 0:

    fig, axs = plt.subplots(num_row_sections, 1, figsize=(7.4, total_row_ht), sharex=True, 
                             sharey=True, gridspec_kw={'wspace':0.05, 'hspace':0.0}) 
    fig.subplots_adjust(top=0.95, bottom=0.2, hspace=0)
    cbar_ax = fig.add_axes([0.15, 0.12, 0.50, 0.025])

    var = make_sect_ar(nrow, num_row_sections)
    sec_list = [(item, slice(None)) for item in var]

    for plot, ax in enumerate(axs):
        section = sec_list[plot]
        xplot = cell_center_x[section]
        plot_sections(ax, section, xplot, 'row', section[0])
        xi = make_plot_coordinates(water_table, bot, cell_center, section, n, xplot.shape[0])
        points, values = get_particle_points(ep_data, section[0], 'Initial Row')
        tt = interpolate_travel_times(points, values, xi)
        im = plot_travel_times(ax, xi[:,0], xi[:,2], tt, (n, ncol))
        anno_sections(ax, num_col_sections, plot)
        anno_age(fig, im)
        
        xline = LineString([Point(xi[0, 0], xi[0, 1]), Point(xi[-1, 0], xi[-1, 1])])
        xs_line = gpd.GeoDataFrame(geometry = gpd.GeoSeries(xline))#, crs=streams.crs)
        xs_line['section'] = 'row {}'.format(section[0])
        gdf = gdf.append(xs_line)

    line = '{}_row_tt_sections.{}'.format(model, form)
    fig_name = os.path.join(fig_dir, line)
    fig.savefig(fig_name)
    #     plt.close()

dst = '{}_xsline.shp'.format(model)
dst_file = os.path.join(fig_dir, dst)
xs_line.to_file(dst_file)

Plot basic cross sections showing water table and model layers

In [ ]:
gb = ep_data.groupby('initial_node_num').median()

idx = pd.Index(np.arange(nlay * nrow * ncol))
gb_array = gb.reindex(idx, fill_value=np.nan)['rt'].values.reshape(nlay, nrow, ncol)
x_array = gb.reindex(idx, fill_value=np.nan)['Initial Geog X'].values.reshape(nlay, nrow, ncol)
y_array = gb.reindex(idx, fill_value=np.nan)['Initial Geog Y'].values.reshape(nlay, nrow, ncol)

dst = os.path.join(fig_dir, '{}.vtu'.format(model))
vtkfile = vt.Vtk(dst, fpmg)
vtkfile.add_array('rtd', gb_array)
vtkfile.write(ibound_filter=True)

In [ ]:
levels = [0, 5, 10, 50, 100, 500, 1000, 5000, 10000]
colors = plt.cm.nipy_spectral(np.linspace(0, 1, len(levels)))
n = 1000 # number of points to interpolate in a vertical column at cell center
form = 'png'
# form = 'pdf'

for lay in layers_to_plot:    
    fig, ax = plt.subplots(1, 1, figsize=(7.4, 7.4))
    ax.set_xlabel('{}'.format(proj_str), ha='right', position=(1.0, 0.1), fontsize=6)
    fig.subplots_adjust(top=0.95, bottom=0.2, hspace=0)
    cbar_ax = fig.add_axes([0.22, 0.15, 0.32, 0.01])

    with np.errstate(invalid='ignore'):
        im = ax.contourf(x_array[lay, :, :], y_array[lay, :, :], gb_array[lay, :, :], 
                         levels=levels, colors=colors, alpha=0.4)

    for shp, c in shapefiles_to_plot:
        src = os.path.dirname(model_ws)
        f = os.path.join(src, shp)
        map_shp = gpd.read_file(f)
        map_shp.plot(ax=ax, color=c, linewidth=1.00, alpha=0.5, **{'edgecolor':'k'})    

    fig.colorbar(im, cax=cbar_ax, orientation='horizontal', alpha=0.5)
    cbar_ax.set_xlabel('Particle travel time in years', y=0.05, rotation=0,  ha='center')

    ax.set_aspect(1)

    line = '{}_median_TT_layer_{}.{}'.format(model, lay, form)
    fig_name = os.path.join(fig_dir, line)
    plt.savefig(fig_name)
    
    line = '{}_median_TT_layer_{}.{}'.format(model, lay, 'tif')
    fig_name = os.path.join(fig_dir, line)

    driver = gdal.GetDriverByName("GTiff")
    dst = driver.Create(fig_name, ncol, nrow, 1, gdal.GDT_Float32)
    dst.SetGeoTransform(gt)
    dst.SetProjection(proj)
    band = dst.GetRasterBand(1)
    band.SetNoDataValue(np.nan)
    band.WriteArray(gb_array[lay, :, :])
    dst = None



In [ ]:
# fig = plt.figure(figsize=(7.4, 12))

# gs = gridspec.GridSpec(10, 10)
# gs.update(wspace=0.0, hspace=0.08, top=0.98, left=0.08, right=0.92)

# ax1 = plt.subplot(gs[0:3, 0:9])
# ax2 = plt.subplot(gs[3:-1, 0:9], sharex=ax1)
# ax3 = plt.subplot(gs[2:7, 9])

# colors_poly = plt.cm.rainbow(np.linspace(0, 1, nlay+1))
# alfa_poly = 0.60
# levels = [0, 10, 50, 100, 500, 10000]

# im = ax1.contourf(xplot * np.ones((n, 1)), pt[:], age[:], colors=colors_poly, alpha=alfa_poly, 
#                   levels=levels, antialiased=True)

# ax1.plot(xplot, land_surface[index, col_to_plot], label='land surface', color='black', lw=0.5)
# ax1.plot(xplot, water_table[index, col_to_plot], label='simulated\nwater table', color='blue', lw=0.75)
# ax1.plot(xplot, bot[num_surf_layers-2, index, col_to_plot], label='bedrock',
#          color='black', linestyle='solid', lw=0.5, alpha=alfa_poly)
# ax1.plot(xplot, bot[num_surf_layers-1, index, col_to_plot], label='bedrock', 
#          color='black', linestyle='solid', lw=0.5, alpha=alfa_poly)
# ax1.plot(xplot, bot[-1, index, col_to_plot], color='black', linestyle='solid', lw=0.5)

# ax1.set_aspect('auto', adjustable='box-forced', anchor='NE')  
# ax1.set_ylabel('Altitude in meters')
# ax1.spines['right'].set_visible(False)
# ax1.spines['top'].set_visible(False)
# ax1.spines['left'].set_visible(True)
# ax1.spines['bottom'].set_visible(False)
# ax1.tick_params('x', length=0)

# im = ax2.contourf(xdat, ydat, ageL, colors=colors_poly, alpha=alfa_poly, levels=levels,
#                  antialiased=True)

# src = os.path.dirname(model_ws)
# fname1 = 'domain_outline.shp'
# f = os.path.join(src, fname1)
# basin = gp.read_file(f)
# basin.plot(ax=ax2, color='none', linewidth=1.00, alpha=1.0, **{'edgecolor':'k'})    

# ax2.plot(xdat[index, col_to_plot], ydat[index, col_to_plot], color='k', lw=1)

# ax2.set_aspect('equal', adjustable='box-forced', anchor='NE')
# ax2.spines['right'].set_visible(False)
# ax2.spines['top'].set_visible(False)
# ax2.spines['left'].set_visible(False)
# ax2.spines['bottom'].set_visible(False)
# ax2.tick_params('both', length=5)

# cbar = fig.colorbar(im, cax=ax3, ax=ax3, orientation='vertical')#, shrink=0.5, pad=0.10, use_gridspec=True) 
# cbar.ax.set_ylabel('Particle travel time / porosity in years', rotation=90, x=0.1, y=0.5, ha='center')
# ax3.set_xmargin(0)
# ax3.set_ymargin(0)
# ax3.set_aspect(15)

# fig.text(0.01, 0.96, 'a', fontdict={'weight':'bold'})
# fig.text(0.01, 0.68, 'b', fontdict={'weight':'bold'})
# fig.text(0.90, 0.12, 'Albers Equal Area meters GRS80', ha='right')

# form_list = ['png', 'pdf']
# for form in form_list:
#     line = 'Paper #2017WR021531-f03_combined.{}'.format(form)
#     fig_name = os.path.join(fig_dir, line)
#     plt.savefig(fig_name, dpi=300)
# # plt.close()

In [ ]:
# experimental

# from matplotlib.transforms import Affine2D
# import mpl_toolkits.axisartist.floating_axes as floating_axes
# import mpl_toolkits.axisartist.angle_helper as angle_helper
# from matplotlib.projections import PolarAxes
# from mpl_toolkits.axisartist.grid_finder import (FixedLocator, MaxNLocator,
#                                                  DictFormatter)

# x, y = clip.loc[0, 'geometry'].exterior.coords.xy
# exts = (x[0], x[2], y[3], y[1])

# src = os.path.join(os.path.dirname(model_ws), 'grid_spec.txt')

# with open(src) as f:
#     lines = f.readlines()

# key = 'Rotation about upper left corner in radians and degrees from positive x axis\n'
# lineno = [item for item in enumerate(lines) if key in item][0][0] + 1
# angle = np.float32(lines[lineno].split()[1])

# def setup_axes1(fig, rect):
#     """
#     A simple one.
#     """
#     tr = Affine2D().scale(2, 1).rotate_deg(angle)

#     grid_helper = floating_axes.GridHelperCurveLinear(
#         tr, extremes=exts)

#     ax1 = floating_axes.FloatingSubplot(fig, rect, grid_helper=grid_helper)
#     fig.add_subplot(ax1)

#     aux_ax = ax1.get_aux_axes(tr)

#     grid_helper.grid_finder.grid_locator1._nbins = 4
#     grid_helper.grid_finder.grid_locator2._nbins = 4

#     return ax1, aux_ax

# fig = plt.figure(1, figsize=(8, 4))
# # fig.subplots_adjust(wspace=0.3, left=0.05, right=0.95)

# ax1, aux_ax1 = setup_axes1(fig, 111)
# # streams.plot(ax=aux_ax1)
# # basin.plot(ax=aux_ax1)
# clip.plot(ax=aux_ax1)
# # fig.gca().set_aspect('equal', adjustable='datalim', anchor='SW')  



In [ ]:
# import osr
# from flopy.utils.reference import SpatialReference

# srs = osr.SpatialReference()
# srs.ImportFromWkt(proj)
# new_proj = srs.ExportToProj4()

# b=''
# for i,j in clip.crs.items():
#     a = '+{}={} '.format(i, j)
#     b += a

# xll, yll = grid_verts[-1, 0], grid_verts[-1, 0] # origin of the model (lower left corner)
# rot = -27.388306963180966 # rotation (positive counterclockwise)

# sr = SpatialReference(delr=delr, delc=delc, xll=xll, yll=yll, rotation=rot, proj4_str=b)
# sr.xcentergrid, sr.ycentergrid

In [ ]:
# dst = '{}_zone{}.tif'.format(model, 15)
# dst_file = os.path.join(fig_dir, dst)

# import gdal
# driver = gdal.GetDriverByName("GTiff")
# dst = driver.Create(dst_file, ncol, nrow, 1, gdal.GDT_Float32)
# dst.SetGeoTransform(gt)
# dst.SetProjection(proj)
# band = dst.GetRasterBand(1)
# band.SetNoDataValue(np.nan)
# band.WriteArray(ageL)
# dst = None

# fig, ax = plt.subplots(1, 1)
# # combine the grid transform with the axis transform
# trans_data = mta + ax.transData

# heads[heads == hdry] = np.nan
# heads[heads == hnoflo] = np.nan

# im = ax.imshow(heads[0, : ,:], transform = trans_data)
# fig.colorbar(im)
# ax.set_xlim(xmin, xmax)
# ax.set_ylim(ymin, ymax)